# Hypothesis testing

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, time
from tqdm.notebook import tqdm_notebook
import glob
import tmdbsimple as tmdb

In [2]:
with open('/Users/nicholasgiuffrida/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
#  Set the API_KEY
tmdb.API_KEY =  login['api-key']

In [4]:
# Create the folder for saving files (if it doesn't exist)
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
# Show the list of files included in the folder
sorted(os.listdir(FOLDER))

['.csv',
 '.ipynb_checkpoints',
 'B_filtered.csv',
 'IMDB Movie Dataset Info.docx',
 'basics',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'final_tmdb_data_2004.csv.gz',
 'final_tmdb_data_2005.csv.gz',
 'final_tmdb_data_2006.csv.gz',
 'final_tmdb_data_2007.csv.gz',
 'final_tmdb_data_2008.csv.gz',
 'final_tmdb_data_2009.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2011.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'final_tmdb_data_2013.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'final_tmdb_data_2017.csv.gz',
 'final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2019.csv.gz',
 'final_tmdb_data_2020.csv.gz',
 'final_tmdb_data_2021.csv.gz',
 'movies db erd.ndm2',
 'movies.db',
 'movies_db.png',
 'movies_db.sql',
 'ratings_filtered.csv',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'tmdb_api_results_2000.jso

In [5]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
        # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
        # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [6]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        #  Load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Set file current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [7]:
YEARS_TO_GET = [2022]
# Create an empty list for saving errors
errors = []

In [8]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
  # create JSON file to store results  
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
   # Check if exists 
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    # save empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
        # If it exists, print message
    else:
        print(f'The file {JSON_FILE} already exists.')

YEARS:   0%|          | 0/1 [00:00<?, ?it/s]

The file Data/tmdb_api_results_2022.json already exists.


In [9]:
basics = pd.read_csv('Data/B_filtered.csv')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
86974,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
86975,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
86976,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
86977,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [10]:
# Filter for movies from selected startYear
df = basics.loc[ basics['startYear']==YEAR].copy()
# Save movie ids to list
movie_ids = df['tconst']
movie_ids.head()

29       tt0119830
37       tt0120589
583      tt0200940
4244     tt0326716
10042    tt0473100
Name: tconst, dtype: object

In [11]:
# Load data from json into a dataframe
previous_df = pd.read_json(JSON_FILE)
previous_df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0119830,0.0,None,None,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",,307046.0,en,One Dog Day,...,0.0,101.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,One Dog Day,0.0,0.000,0.0,
2,tt0120589,0.0,/1ZilEjqou6JC369GOKwHWeqCbYL.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}]",,649985.0,pt,Predestinado: Arigó e o Espírito do Dr. Fritz,...,0.0,100.0,"[{'english_name': 'Portuguese', 'iso_639_1': '...",Released,,Predestinado: Arigó e o Espírito do Dr. Fritz,0.0,7.500,12.0,NaN
3,tt0200940,0.0,/qSHvhAEnomjwDNmYVnQIBbQyqpA.jpg,None,0.0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",,68240.0,en,Over-sexed Rugsuckers from Mars,...,0.0,87.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,You'll never trust your vacuum cleaner again!,Over-sexed Rugsuckers from Mars,0.0,3.300,4.0,NaN
4,tt0326716,0.0,/oLyQu5QAR1w7e8roeveNiZXS5kN.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",http://5-25-77themovie.com,199734.0,en,5-25-77,...,0.0,132.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The true story of a boy... a girl... and the m...,5-25-77,0.0,6.200,8.0,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,tt15422244,0.0,/fbNi8ygJ2w3AljcSWFDKrfWuGxV.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}]",https://www.sonyliv.com/movies/saudi-vellaka-m...,873992.0,ml,സൗദി വെള്ളക്ക,...,0.0,146.0,"[{'english_name': 'Malayalam', 'iso_639_1': 'm...",Released,,Saudi Vellakka,0.0,7.800,6.0,NaN
1723,tt15422276,0.0,/ntGzo9lNetur0nUtVjJj39qn7a.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,942393.0,pa,Main Viyah Nahi Karona Tere Naal,...,0.0,0.0,"[{'english_name': 'Punjabi', 'iso_639_1': 'pa'...",Released,,Main Viyah Nahi Karona Tere Naal,0.0,7.000,1.0,NaN
1724,tt15426246,0.0,/6KqccyEBwLJZtbF0lJCT1tEu8zB.jpg,None,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 27, 'n...",http://www.vmiworldwide.com/index.php/titles/d...,1028955.0,en,In Isolation,...,0.0,87.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,In Isolation,0.0,3.929,7.0,
1725,tt15426294,0.0,/1179VQ6ckccWzgyS2PHxWMFDGKI.jpg,None,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",,874355.0,en,Gone in the Night,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some secrets are better left buried.,Gone in the Night,0.0,5.540,99.0,R


In [12]:
# filter any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [13]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
  # create JSON file to store results  
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
   # Check if exists 
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    # save empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
        # If it exists, print message
    else:
        print(f'The file {JSON_FILE} already exists.')

YEARS:   0%|          | 0/1 [00:00<?, ?it/s]

The file Data/tmdb_api_results_2022.json already exists.


In [14]:
# Save the final results to a csv.gz file
final_year_df = pd.read_json(JSON_FILE)

csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
final_year_df.to_csv(csv_fname, compression="gzip", index=False)

In [ ]:
YEARS_TO_GET = [2021]
# Create an empty list for saving errors
errors = []

In [ ]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
  # create JSON file to store results  
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
   # Check if exists 
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    # save empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
        # If it exists, print message
    else:
        print(f'The file {JSON_FILE} already exists.')

In [ ]:
# Filter for movies from selected startYear
df = basics.loc[ basics['startYear']==YEAR].copy()
# Save movie ids to list
movie_ids = df['tconst']
movie_ids.head()

In [ ]:
# Load data from json into a dataframe
previous_df = pd.read_json(JSON_FILE)
previous_df

In [ ]:
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [ ]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
  # create JSON file to store results  
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
   # Check if exists 
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    # save empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
        # If it exists, print message
    else:
        print(f'The file {JSON_FILE} already exists.')

In [ ]:
# Save the final results to a csv.gz file
final_year_df = pd.read_json(JSON_FILE)

csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
final_year_df.to_csv(csv_fname, compression="gzip", index=False)